https://colab.research.google.com/drive/1T88YY-c2t37z73E_Qo_u1NBvyjy3kEim?usp=sharing#scrollTo=HU0KTERNllyB

https://towardsdatascience.com/how-to-perform-topic-modeling-with-top2vec-1ae9bb4e89dc

In [1]:
!pip install --upgrade llvmlite
!pip install --upgrade git+https://github.com/scikit-learn-contrib/hdbscan.git#egg=hdbscan
!pip install top2vec  # easy way to install Top2Vec
!pip install top2vec[sentence_encoders]  # install if using pre-trained universal sentence encoder options
!pip install top2vec[sentence_transformers]  # install if using pre-trained BERT sentence transformer options
!pip install git+https://github.com/MartinoMensio/spacy-universal-sentence-encoder.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/scikit-learn-contrib/hdbscan.git to /tmp/pip-install-dgnirk7w/hdbscan_1b0be07f5058447c9cbc5d8401c7d27c
  Running command git clone --filter=blob:none --quiet https://github.com/scikit-learn-contrib/hdbscan.git /tmp/pip-install-dgnirk7w/hdbscan_1b0be07f5058447c9cbc5d8401c7d27c
  Resolved https://github.com/scikit-learn-contrib/hdbscan.git to commit e55f957441fa58c109971fb868a399b498578234
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp39-cp39-linux_x86_64.whl size=3582036 sha256=05bd3518394bf4addd40f7f120cf7e4f8aaa3cb6d91affa6cea5e81419f3e640
  Stored in directory: /tmp/pip-ephem-wheel-cache-_a2a86qx/wheels/d9/a3/52/1cab908

In [2]:
!pip install pandas==1.5.3
import pandas as pd

from top2vec import Top2Vec
import pandas as pd
import numpy as np
import multiprocessing
import time
import csv
import io

multiprocessing.cpu_count()


from google.colab import drive
#import os
!pip install pandas==1.5.3
#!pip install pandas==1.3.5
import pandas as pd

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/MITB_AI/Text analytics dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 65.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/gdrive
/content/gdrive/MyDrive/MITB_AI/Text analytics dataset


In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [3]:
%ls

posts_after_lemmatized_non_suicidal.pkl
posts_suicidal_with_tfidf2.pkl
Top2VecModel_USE_X_Train.jl
Top2VecModel_USE_X_Train_nonSuicidal.jl
Top2VecModel_X_Train.jl
Top2VecModel_X_Train_nonSuicidal.jl
Top2Vec_USE_X_Train_nonSuicidal_TopicsOutput.csv
Top2Vec_USE_X_Train_TopicsOutput.csv
Top2Vec_X_Train_nonSuicidal_TopicsOutput.csv
Top2Vec_X_Train_TopicsOutput.csv
X_Train_nonSuicidal.pkl
X_Train.pkl


In [4]:
# df2 = pd.read_csv(io.BytesIO(uploaded['Suicide_Detection_non-suicide only_processed.csv']))
#df2 = pd.read_csv('Suicide_Detection_suicide only_processed_small.csv', engine='python', error_bad_lines=False, encoding='ISO-8859-1')
#print(len(df2))
#print(df2.head())

#print(y_train.head())

X_train = pd.read_pickle("X_Train.pkl")

from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # min occurrences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=False,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

df_filtered_lemmatized = X_train["text_lemmatized"].tolist()

stop_list = ("suicidal", "suicide", "aah", "fuck")
df_filtered_lemmatized_removeSuicide = [[w for w in doc if w not in stop_list] for doc in df_filtered_lemmatized]
df_filtered_lemmatized_joined = [' '.join(x) for x in df_filtered_lemmatized_removeSuicide]  # joined to fit CountVectorizer
train1_vecs = vectorizer.fit_transform(df_filtered_lemmatized_joined) #may be needed for bertopic
feature_names = vectorizer.get_feature_names_out()
# display the first 20 tokens
feature_names[:20]

array(['aaron', 'aba', 'aback', 'abandon', 'abandoned', 'abandonment',
       'abdomen', 'abdominal', 'abhor', 'abhorrent', 'abide', 'ability',
       'abject', 'able', 'abnormal', 'abnormality', 'abnormally',
       'abomination', 'abort', 'aborted'], dtype=object)

In [5]:
#df2_list = [item for sublist in df2.values.tolist() for item in sublist]
print(train1_vecs.shape)


(81225, 11314)


In [ ]:
start_time = time.time()
#model = Top2Vec(documents=df_filtered_lemmatized_joined)
#model = Top2Vec(documents=df_filtered_lemmatized_joined, embedding_model='universal-sentence-encoder')
model = Top2Vec(documents=df_filtered_lemmatized_joined, speed="deep-learn", workers=multiprocessing.cpu_count())  # uncomment to use Doc2Vec
print('Total time taken (mins): ', int((time.time()-start_time)/60))

2023-03-12 13:14:15,216 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2023-03-12 13:14:32,960 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding


In [ ]:
#topic_words, word_scores, topic_nums = model.get_topics(num_topics=len(model.get_topic_sizes()))
topic_words, word_scores, topic_nums = model.get_topics()


print(topic_words)
print(len(topic_words))

In [ ]:
#topic_words, _, topic_docs = model.get_topics(num_topics=len(model.get_topic_sizes()))
for i in range(len(topic_words)):
    print(f"Topic {i}: {' | '.join(topic_words[i])}")
    #print(f"Documents: {', '.join(topic_docs[i])}\n")

import joblib
joblib.dump(model, 'Top2VecModel_D2V_X_Train.jl')


In [ ]:
import joblib
model = joblib.load('Top2VecModel_D2V_X_Train.jl')
#nmf_output2 = joblib.load('suicidal_nmf_output2.jl')

In [ ]:
model.hierarchical_topic_reduction(num_topics=10)
model.topic_words_reduced[0]

In [ ]:
topic_sizes, topic_nums = model.get_topic_sizes()
print(topic_sizes[:10])  # get sizes of top 10 topics
np.sum(topic_sizes)

In [ ]:

#topics, word_scores, topic_nums = model.get_topics(10)
topics, word_scores, topic_nums = model.get_topics()

# topic_words, word_scores, topic_nums = test.get_topics(test.get_num_topics())
# for topic in topic_nums[:5]:
#     test.generate_topic_wordcloud(topic)

In [ ]:
topic_keywords = model.topic_words_reduced

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
df_topic_keywords.to_csv('Top2Vec_D2V_X_Train_TopicsOutput.csv')